# Basic DTW Code
Code from https://towardsdatascience.com/dynamic-time-warping-3933f25fcdd  
This code implements a simple DTW measure between two series represented as lists.  
It allows for series of different lengths and has a `window` parameter that determines the amount of warping allowed.  
For series of different lengths, the minimum warping will be the difference in the lengths.  

In [1]:
import numpy as np

In [2]:
#input two arrays and a warping metric, output distance between arrays
def dtw(s, t, window):
    n, m = len(s), len(t)
    w = np.max([window, abs(n-m)]) # warping cannot be less than the difference in lengths. 
    dtw_matrix = np.zeros((n+1, m+1))
    
    for i in range(n+1):
        for j in range(m+1):
            dtw_matrix[i, j] = np.inf
    dtw_matrix[0, 0] = 0
    
    for i in range(1, n+1):
        for j in range(np.max([1, i-w]), np.min([m, i+w])+1):
            dtw_matrix[i, j] = 0
    
    for i in range(1, n+1):
        for j in range(np.max([1, i-w]), np.min([m, i+w])+1):
            cost = abs(s[i-1] - t[j-1])
            # take last min from a square box
            last_min = np.min([dtw_matrix[i-1, j], dtw_matrix[i, j-1], dtw_matrix[i-1, j-1]])
            dtw_matrix[i, j] = cost + last_min
    return dtw_matrix[-1,-1]

In [3]:
a = [1,2,3,3,5]
b = [1,2,2,2,2,2,2,4]
dtw(a,b, window = 3)

3.0

In [4]:
x1 = [7,7,8,9,10,10,7,4,2,1,2,4,7,11,10,9,7]
x2 = [7,8,10,10,8,7,3,2,2,4,6,12,12,9,7,7]

In [5]:
dtw_0 = dtw(x1,x2,window = 1)
dtw_0

18.0

Works also for numpy arrays.

In [6]:
x = np.array([[7,7,8,9,10,10,7,4,2,1,2,4,7,11,10,9,7],
                [7,8,10,10,8,7,3,2,2,4,6,12,12,9,7,7,8]])

In [7]:
x[0]

array([ 7,  7,  8,  9, 10, 10,  7,  4,  2,  1,  2,  4,  7, 11, 10,  9,  7])

In [8]:
for w in range(10):
    dtw_n = dtw(x[0],x[1],window = w)
    print(w, dtw_n)

0 43.0
1 19.0
2 9.0
3 9.0
4 9.0
5 9.0
6 9.0
7 9.0
8 9.0
9 9.0


# Assignment 1

## Task 1
#### 1. Use the DTW function in the notebook to implement a simple 1-NN classifier for time-series data. 

In [9]:
def predicted_classification(train_data_features, train_data_labels, test_row, window): 
    
    test_row_list = test_row.values.flatten().tolist()
    
    distances = {}
    
    for row_num in train_data_features.index:
        row = train_data_features.loc[row_num].values.flatten().tolist()
        
        dist = dtw(test_row_list, row, window)
        distances[dist] = row_num
        
    nearest_neighbor = distances[min(distances.keys())]
    
    classification = train_data_labels.loc[nearest_neighbor]
    
    return classification

#### 2. Test the performance of this classifier on the dataset provided in the file UMD_TEST.txt. This is a three-class synthetic time-series dataset - see details here https://www.timeseriesclassification.com/description.php?Dataset=UMD.


In [10]:
import pandas as pd
df = pd.read_csv("UMD_TEST.txt", sep="\s+", header= None)

In [11]:
#split test and train data
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

y = df[0]
X = df[list(range(1,144))]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [42]:
#holdout test
window = 1
results2 = []

for example_index in X_test.index:
    actual_label = y_test[example_index]
    predicted_label = predicted_classification(X_train, y_train, X_test.loc[example_index], window)
    results2.append(predicted_label)

In [43]:
#calculate accuracy score
from sklearn.metrics import accuracy_score
y_pred = results2
y_true = y_test
score = accuracy_score(y_true, y_pred)
print("Accuray: {}".format(score))

Accuray: 0.896551724137931


In [15]:
#calculate confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_true, y_pred)

array([[ 8,  1,  0],
       [ 0, 11,  0],
       [ 0,  2,  7]])

This model seems to perform fairly well at classifying the data into the three classes.

#### 3. Given that the time-series are all the same length (150 ticks) Euclidean distance can also be used as distance metric. Compare the 1-NN DTW performance with 1-NN Euclidean. Use the scikit-learn implementation for the Euclidean model. 

In [38]:
#set up euclidean knn
from sklearn.neighbors import KNeighborsClassifier

euclidean_1NN = KNeighborsClassifier(n_neighbors=1, metric="euclidean")
euclidean_1NN.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='euclidean',
                     metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                     weights='uniform')

In [39]:
y_pred = euclidean_1NN.predict(X_test)

print("Acurracy: ", accuracy_score(y_true,y_pred))
confusion_matrix(y_test, y_pred)

Acurracy:  0.7931034482758621


array([[8, 1, 0],
       [1, 9, 1],
       [1, 2, 6]])

The 1NN model with euclidean distance seems to perform worse than the 1NN model with dynamic time warping.

#### 4. Find the best value for the window parameter for this dataset.

In [31]:
def get_acurracy(window):
    results = []

    for example_index in X_test.index:
        actual_label = y_test[example_index]
        predicted_label = predicted_classification(X_train, y_train, X_test.loc[example_index], window)
        results.append(predicted_label)
        
    y_pred = results
    y_true = y_test
    score = accuracy_score(y_true, y_pred)
    
    return score

In [41]:
for window in range(0, 10, 2):
    acc = get_acurracy(window)
    print("Accuracy of {}: {}".format(window, acc))

Accuracy of 0: 0.8275862068965517
Accuracy of 2: 0.896551724137931
Accuracy of 4: 0.9310344827586207
Accuracy of 6: 0.9310344827586207
Accuracy of 8: 0.9310344827586207


In [32]:
get_acurracy(30)

0.9310344827586207

Accuracy rises when increasing the window size between window=0 and window=4, but does not seem to improve further when increasing the size of the window beyond window=4.

## Task 2

#### 1. The k-NN classifier in scikit-learn has a metric parameter that allows for user-defined distance metrics. Adapt the DTW code provided so that it can be incorporated  in a scikit-learn k-NN  classifier. 


In [51]:
def modified_dtw(s, t):
    
    window = 1 #using this value that's the window parameter when running the original dtw in Tsk 1
    
    n, m = len(s), len(t)
    w = np.max([window, abs(n-m)])  
    dtw_matrix = np.zeros((n+1, m+1))
    
    for i in range(n+1):
        for j in range(m+1):
            dtw_matrix[i, j] = np.inf
    dtw_matrix[0, 0] = 0
    
    for i in range(1, n+1):
        for j in range(np.max([1, i-w]), np.min([m, i+w])+1):
            dtw_matrix[i, j] = 0
    
    for i in range(1, n+1):
        for j in range(np.max([1, i-w]), np.min([m, i+w])+1):
            cost = abs(s[i-1] - t[j-1])
            # take last min from a square box
            last_min = np.min([dtw_matrix[i-1, j], dtw_matrix[i, j-1], dtw_matrix[i-1, j-1]])
            dtw_matrix[i, j] = cost + last_min
    return dtw_matrix[-1,-1]

#### 2. Test the performance of this classifier and compare with the 1-NN results from Task 1. Verify that the 1-NN results are consistent. 

In [56]:
#verifying results at k=1 are the same
window = 1
dtw_kNN = KNeighborsClassifier(n_neighbors=1, metric=modified_dtw)
dtw_kNN.fit(X_train,y_train)

y_pred = dtw_kNN.predict(X_test)

print("Acurracy: ", accuracy_score(y_true,y_pred))

Acurracy:  0.896551724137931


1NN using modified_dtw the same acurracy as when using the original dtw function.


In [58]:
confusion_matrix(y_true, y_pred)

array([[ 8,  1,  0],
       [ 1, 10,  0],
       [ 2,  4,  3]])

In [55]:
#modified_dtw with even larger k
dtw_kNN = KNeighborsClassifier(n_neighbors=10, metric=modified_dtw)
dtw_kNN.fit(X_train,y_train)

y_pred = dtw_kNN.predict(X_test)
print("Acurracy: ", accuracy_score(y_true,y_pred))

Acurracy:  0.7931034482758621


Acurracy seems to decrease as the number of neighbors increases.

#### 3. Compare with k-NN Euclidean. 

In [57]:
#modified_dtw with k=4
dtw_kNN = KNeighborsClassifier(n_neighbors=4, metric=modified_dtw)
dtw_kNN.fit(X_train,y_train)

y_pred = dtw_kNN.predict(X_test)
print("Acurracy: ", accuracy_score(y_true,y_pred))
confusion_matrix(y_true, y_pred)

Acurracy:  0.7241379310344828


array([[ 8,  1,  0],
       [ 1, 10,  0],
       [ 2,  4,  3]])

In [59]:
#euclidean distance with k=4
kNN = KNeighborsClassifier(n_neighbors=4, metric="euclidean")
kNN.fit(X_train,y_train)

print("Acurracy: ", accuracy_score(y_true,y_pred))
confusion_matrix(y_true, y_pred)

Acurracy:  0.7241379310344828


array([[ 8,  1,  0],
       [ 1, 10,  0],
       [ 2,  4,  3]])

4NN using euclidean distance seems to perform about the same as 4NN using dynamic time warping.